In [1]:
#If packages not installed
'''
%pip install numpy
%pip install ffmpeg-python
%pip install scipy
%pip install opencv-python
%pip install scikit-image
%pip install PyWavelets
'''

'\n%pip install numpy\n%pip install ffmpeg-python\n%pip install scipy\n%pip install opencv-python\n%pip install scikit-image\n%pip install PyWavelets\n'

In [1]:
import ffmpeg
import numpy as np

In [2]:
class Image:

    y = 0
    u = 0
    v = 0

    def __init__(self, r, g, b):
        self.r = r
        self.g = g
        self.b = b

    def rgb_to_yuv(self):
        self.y = (0.257 * self.r) + (0.504 * self.g) + (0.098 * self.b) + 16
        self.u = (-0.148 * self.r) - (0.291 * self.g) + (0.439 * self.b) + 128
        self.v = (0.439 * self.r) - (0.368 * self.g) - (0.071 * self.b) + 128
    
    def yuv_to_rgb(self):
        self.b = 1.164 * (self.y - 16) + 2.018 * (self.u - 128)
        self.g = 1.164 * (self.y - 16) - 0.813 * (self.v - 128) - 0.391 * (self.u - 128)
        self.r = 1.164 * (self.y - 16) + 1.596 * (self.v - 128) 
    
    def print_RGB_values(self):
        print("R:", self.r, "\n")
        print("G:", self.g, "\n")
        print("B:", self.b, "\n")

    def print_YUV_values(self):
        print("Y:", self.y, "\n")
        print("U:", self.u, "\n")
        print("V:", self.v, "\n")

**Exercise 2**
---

In [3]:
#Exercise 2
i1 = Image(100, 50, 255)
i1.print_RGB_values()
i1.rgb_to_yuv()
i1.print_YUV_values()
i1.yuv_to_rgb()
i1.print_RGB_values()

R: 100 

G: 50 

B: 255 

Y: 91.89 

U: 210.59500000000003 

V: 135.395 

R: 100.13838000000001 

G: 50.02917999999999 

B: 255.01267000000004 



**Exercise 3**
---

In [6]:

out=(
    ffmpeg
    .input('image.jpg')
    .filter("scale", 1080,1080)
    .output('out_exercise3.jpg')
    .run(capture_stdout=True)
)


**Exercise 4**
---

In [ ]:
#Code by 张家琪 Zhang Jiaqi taken from https://dev.to/marycheung021213/understanding-dct-and-quantization-in-jpeg-compression-1col

# Obtaining a mask through zigzag scanning
def z_scan_mask(C,N):
    mask=np.zeros((N,N))
    start=0
    mask_m=start
    mask_n=start
    for i in range(C):
        if i==0:
            mask[mask_m,mask_n]=1
        else:
            # If even, move upward to the right
            if (mask_m+mask_n)%2==0:
                mask_m-=1
                mask_n+=1
                # If it exceeds the upper boundary, move downward
                if mask_m<0:
                    mask_m+=1
                # If it exceeds the right boundary, move left
                if mask_n>=N:
                    mask_n-=1
            # If odd, move downward to the left
            else:
                mask_m+=1
                mask_n-=1
                # If it exceeds the lower boundary, move upward
                if mask_m>=N:
                    mask_m-=1
                # If it exceeds the left boundary, move right
                if mask_n<0:
                    mask_n+=1
            mask[mask_m,mask_n]=1
    return mask

In [ ]:
'''
path = 'coltrane.jpg'
coltrane = ffmpeg.input(path)

probe = ffmpeg.probe(path)

image_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')

width = int(image_info['width'])
height = int(image_info['height'])

out, err = (
    ffmpeg
    .input(path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run(capture_stdout=True)
)

coltrane_array = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)
'''

In [ ]:
'''
image_squeezed = np.squeeze(coltrane_array)

r = image_squeezed[:, :, 0]
g = image_squeezed[:, :, 1]
b = image_squeezed[:, :, 2]
'''

**Exercise 5.1**
---

In [ ]:
# Exercise 5.1:
path = 'superbowl.jpg'
coltrane = ffmpeg.input(path)

probe = ffmpeg.probe(path)

image_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')

width = int(image_info['width'])
height = int(image_info['height'])


ffmpeg.output(
    coltrane.filter('hue', s=0),
    'output_exercise_5.jpg'
).run()



(None, None)

In [ ]:
#We assumed that the chain is given in an array. like: [1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1]

def codif_bits(bits_chain):
    length = len(bits_chain)
    encoding = ""
    i = 0
    while i < length:
        count = 1
        while i + 1 < length and bits_chain[i] == bits_chain[i + 1]:
            count += 1
            i += 1

        encoding += str(bits_chain[i]) + str(count)
        i += 1
    
    return encoding



In [ ]:
bits = [1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1] 
encoded_bits = codif_bits(bits) 
print(encoded_bits)

1403110212


**Exercise 6**
---

In [7]:
import cv2
from scipy.fftpack import dct, idct
import skimage.io as io

In [ ]:
#Create the class
class DCT:
    
    def __init__(self, img):
        self.image = img
    

    
    #Code by 张家琪 Zhang Jiaqi taken from https://dev.to/marycheung021213/understanding-dct-and-quantization-in-jpeg-compression-1col

    # overlaying the mask, discarding the high-frequency components
    def compressDCT(self, mask,N):
        self.image = np.float32(self.image)
        self.img_dct=np.zeros((self.image.shape[0]//N*N,self.image.shape[1]//N*N))
        for m in range(0,self.img_dct.shape[0],N):
            for n in range(0,self.img_dct.shape[1],N):
                block=self.image[m:m+N,n:n+N]
                # DCT
                coeff=cv2.dct(block)
                # IDCT, but only the parts of the image where the mask has a value of 1 are retained
                iblock=cv2.idct(coeff*mask)
                self.img_dct[m:m+N,n:n+N]=iblock

In [ ]:
#Importing and pre-processing
p = "neptunes.jpg"
img=io.imread(p,as_gray=True)
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
DCTimg = DCT(img)

In [ ]:
C = 10
N = 8
mask = z_scan_mask(C, N)

In [ ]:
#Apply DCT
DCTimg.compressDCT(mask, N)
cv2.imshow("image DCT", DCTimg.img_dct)
cv2.waitKey(0)
cv2.destroyAllWindows()

**Exercise 7**
---

In [ ]:
import pywt

In [ ]:
class DWT:
    def __init__(self, image):
        self.image = image
    
    #Code by 张家琪 Zhang Jiaqi taken from https://dev.to/marycheung021213/understanding-dct-and-quantization-in-jpeg-compression-1col
    #modified by Junior Bernales

    # overlaying the mask, discarding the high-frequency components
    def compressDWT(self, N):
        self.image = np.float32(self.image)
        self.img_dwt=np.zeros((self.image.shape[0]//N*N,self.image.shape[1]//N*N))
        for m in range(0,self.img_dwt.shape[0],N):
            for n in range(0,self.img_dwt.shape[1],N):
                block=self.image[m:m+N,n:n+N]
                # DCT
                coeff=pywt.dwt2(block, 'db1')
                # IDCT, but only the parts of the image where the mask has a value of 1 are retained
                iblock=pywt.idwt2(coeff, 'db1')
                self.img_dwt[m:m+N,n:n+N]=iblock